# 第二章：语音特征提取
原始的语音信号有过多冗杂信息，直接输入机器学习模型的效率较低，故需要进行语音信号的预处理，并提取语音特征。

## 预加重
语音经发声者的口唇辐射发出，会受到唇端辐射抑制，高频能量被明显降低，
语音信号的频率提高2倍时，功率谱的幅度下降6dB.即语音信号的高频部分受到的抑制较大。
预加重就是为了补偿语音信号高频部分的振幅。

公式：
```
s`(n)=s(n)-a*s(n-1)  a≈0.98
```

In [ ]:
from copy import deepcopy
from collections.abc import Container
def preemphasis(data, a=0.98):
    """预加重，公式:s`(n)=s(n)-a*s(n-1)  a≈0.98

    Args:
        data (List[int]): 待加重的语音数据，一维数组
        a (float, optional): 预加重系数. Defaults to 0.98.

    Returns:
        rst: List[float] 加重结果，使用了deepcopy，不会修改输入的data.
    """
    # 要求输入的是一维数组(即元素不是容器类型)
    assert all(not isinstance(item, Container) for item in data)
    length = len(data)
    rst = deepcopy(data)
    for i in range(1, length):
        rst[i] = rst[i] - a * rst[i - 1]
    return rst

## 分帧
将data分为多个frame_length长度的二维数组，帧与帧之间存在重叠，故指定跳数hop_size，一般要求hop_size小于frame_length/2  
语音信号是一个短时平稳信号，浊音是有规律的声带振动，即基音频率在短时范围内相对稳定。  
可以认为，10~30ms内的语音片段是一个准稳态的，分为一帧。  
一帧常规定为20ms~25ms，在采样率16Khz下，25ms意味着400个采样点。  
两帧之间的基音可能变化，故重叠取帧，帧移10ms，重叠50%~60%。  

In [ ]:
import numpy as np
def framing(data, frame_length=2048, hop_size=512):
    """分帧，将data分为多个frame_length长度的二维数组，
        帧与帧之间存在重叠，故指定跳数hop_size，一般要求hop_size<frame_length/2

    Args:
        data (List[float]): 预加重的音频数据
        frame_length (int, optional): 每帧长. Defaults to 2048.
        hop_size (int, optional): 步长. Defaults to 512.

    Returns:
        rst: List[List[float]] 分帧结果
    """
    # 要求输入的是一维数组
    assert all(not isinstance(item, Container) for item in data)
    singal_length = len(data)
    # 计算帧数
    # 假设100个点，帧长20，每次移动10，求分几帧？
    # 1-20, 10-30, 20-40...80-100 显然是8帧，计算方法是(采样点数-帧长)/step
    fn = (singal_length - frame_length) / hop_size
    # 向上取整，以保留所有帧
    fn = int(np.ceil(fn))
    # 如果有105个点,求补充0的数量? 多走一帧的开始:fn*step，+frame_length成为结尾，减去原采样点的数量得解。
    # 80-100, 90-110，补充5个0. 计算方法是(fn*step+frame_length)-singal_length
    zero_nums = (fn * hop_size + frame_length) - singal_length
    zero_array = np.zeros(zero_nums)
    # 拼接
    tmp_data = np.array(deepcopy(data))
    zero_array = zero_array.astype(tmp_data.dtype)
    tmp_data = np.concatenate((tmp_data, zero_array))
    # np.tile(A, times) 把数字A重复times次，times可以是高维的,本方法很好用.
    rst = np.array(
        [tmp_data[index : index + frame_length] for index in range(fn)]
    ).astype(tmp_data.dtype)
    return rst

## 加窗
分帧相当于对信号加了矩形窗，会发生过强的频谱泄露，加汉宁或汉明窗，以减少频谱泄露。